In [1]:
import json
from datetime import datetime, timedelta
import subprocess

In [2]:
lsg_web_repo = "../../lsg/"
now = datetime.now()

years = [str(y) for y in range(2016, now.year + 1)]
months = ["0" + str(m) for m in range(2, 9)]

def process(cmd):
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.strip().decode('utf-8'), result.stderr.strip().decode('utf-8')

def count_players(text):
    lines = text.split('\n')
    return sum(1 for line in lines if line.startswith("- "))

def get_zarejestrowani(date):
    rev_out, rev_err = process([
        "git", "-C", lsg_web_repo, "rev-list", "-1", f"--before={date}T00:00", "HEAD"
    ])

    if rev_out:
        zar_out, zar_err = process([
            "git", "-C", lsg_web_repo, "show", f"{rev_out}:./zarejestrowani.md"
        ])
        if zar_out:
            return zar_out
    return ""

In [3]:
zarejestrowani = {}
for year in years:
    zarejestrowani[year] = {}

    start_date = datetime(int(year), 2, 1)
    end_date = datetime(int(year), 8, 20)
    dates = [(start_date + timedelta(days=x)) for x in range((end_date - start_date).days)]

    for date in dates:
        if int(date.year) >= now.year and int(date.month) >= now.month and int(date.day) >= now.day:
            break # no looking into the future
        strdate = date.strftime('%Y-%m-%d')
        zar = get_zarejestrowani(strdate)
        cnt = count_players(zar)
        if int(date.month) > 6 and cnt < 20:
            break # no artifacts due to early reset
        zarejestrowani[str(date.year)][date.strftime('%m-%d')] = cnt
        #print(f"{strdate} {cnt:4.0f}")

with open('zarejestrowani.json', 'w') as f:
    json.dump(zarejestrowani, f, indent=2)